### Big thanks to Susan Li for the awesome tutorial on LDA
[https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24]

This Notebook was inspired by the Medium article above and is written by Advith Chegu for HackDown 2020. Some more sources:

[https://medium.com/@osas.usen/topic-extraction-from-tweets-using-lda-a997e4eb0985]

[https://www.kaggle.com/therohk/million-headlines/data]

In [39]:
import pandas as pd
DATASET_COLUMNS  = ["sentiment", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding=DATASET_ENCODING , names=DATASET_COLUMNS)
data_text = data[['text']]
data_text.set_index('text', drop=True, append=False, inplace=False, verify_integrity=False)
data_text['index'] = data_text.index
documents = data_text

/Users/advithchegu/.local/share/virtualenvs/twitter-dash--SggTynj/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [40]:
print(len(documents))
print(documents[:5])

1600000
                                                text  index
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...      0
1  is upset that he can't update his Facebook by ...      1
2  @Kenichan I dived many times for the ball. Man...      2
3    my whole body feels itchy and like its on fire       3
4  @nationwideclass no, it's not behaving at all....      4


In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/advithchegu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [46]:
stemmer = SnowballStemmer("english")
stop_words = ['http']

In [49]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words and '@' not in token:
            result.append(lemmatize_stemming(token))
    return result

In [50]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['explain', 'how,', 'jazmine', 'got', 'from', 'http://is.gd/raX6', 'to', 'http://is.gd/raWM.', '', 'perhaps', 'im', 'being', 'elitist', 'but', 'its', 'so', 'disappointing', '']


 tokenized and lemmatized document: 
['explain', 'jazmin', 'rawm', 'elitist', 'disappoint']


In [52]:
processed_docs = documents['text'].map(preprocess)
processed_docs[:10]

0    [switchfoot, twitpic, awww, bummer, shoulda, d...
1    [upset, updat, facebook, text, result, school,...
2    [kenichan, dive, time, ball, manag, save, rest...
3                            [bodi, feel, itchi, like]
4                             [nationwideclass, behav]
5                                     [kwesidei, crew]
6                                               [need]
7            [loltrish, long, time, rain, fine, thank]
8                                    [tatiana_k, nope]
9                                    [twittera, muera]
Name: text, dtype: object

In [15]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [16]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [17]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3567, 1), (3568, 1)]

In [18]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3567 ("compulsori") appears 1 time.
Word 3568 ("ratepay") appears 1 time.


In [19]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5850076620505259),
 (1, 0.38947256567331934),
 (2, 0.4997099083387053),
 (3, 0.5063271308533074)]


In [20]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [21]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"hous" + 0.022*"south" + 0.019*"north" + 0.017*"bushfir" + 0.016*"miss" + 0.013*"interview" + 0.012*"west" + 0.012*"hospit" + 0.011*"island" + 0.011*"coast"
Topic: 1 
Words: 0.032*"kill" + 0.023*"shoot" + 0.021*"protest" + 0.021*"dead" + 0.020*"polic" + 0.019*"attack" + 0.014*"offic" + 0.014*"assault" + 0.013*"chines" + 0.011*"michael"
Topic: 2 
Words: 0.057*"australia" + 0.046*"australian" + 0.026*"world" + 0.018*"canberra" + 0.017*"test" + 0.013*"win" + 0.011*"final" + 0.011*"farm" + 0.010*"return" + 0.009*"beat"
Topic: 3 
Words: 0.030*"polic" + 0.029*"charg" + 0.026*"court" + 0.024*"death" + 0.024*"murder" + 0.020*"woman" + 0.017*"face" + 0.016*"alleg" + 0.015*"crash" + 0.013*"trial"
Topic: 4 
Words: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"climat" + 0.012*"concern" + 0.011*"flood" + 0.010*"fear" + 0.010*"emerg"
Topic: 5 
Words: 0.021*"market" + 0.020*"news" + 0.018*"women" + 0.018*"live" + 0.016*"tasmania" + 0.013*

In [22]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [23]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.45240768790245056	 
Topic: 0.020*"donald" + 0.014*"farmer" + 0.014*"nation" + 0.013*"time" + 0.013*"rural" + 0.012*"council" + 0.012*"indigen" + 0.012*"school" + 0.011*"plan" + 0.011*"commiss"

Score: 0.44756072759628296	 
Topic: 0.031*"govern" + 0.019*"feder" + 0.016*"warn" + 0.015*"countri" + 0.015*"fund" + 0.015*"claim" + 0.014*"life" + 0.013*"health" + 0.012*"say" + 0.012*"stori"

Score: 0.012506908737123013	 
Topic: 0.035*"elect" + 0.018*"water" + 0.018*"state" + 0.016*"tasmanian" + 0.013*"labor" + 0.011*"liber" + 0.011*"morrison" + 0.011*"leader" + 0.011*"parti" + 0.010*"campaign"

Score: 0.012503824196755886	 
Topic: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"climat" + 0.012*"concern" + 0.011*"flood" + 0.010*"fear" + 0.010*"emerg"

Score: 0.012503472156822681	 
Topic: 0.022*"hous" + 0.022*"south" + 0.019*"north" + 0.017*"bushfir" + 0.016*"miss" + 0.013*"interview" + 0.012*"west" + 0.012*"hospit" + 0.011*"island" + 0.011*"coast

In [27]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4523957371711731	 
Topic: 0.020*"donald" + 0.014*"farmer" + 0.014*"nation" + 0.013*"time" + 0.013*"rural" + 0.012*"council" + 0.012*"indigen" + 0.012*"school" + 0.011*"plan" + 0.011*"commiss"

Score: 0.4475727081298828	 
Topic: 0.031*"govern" + 0.019*"feder" + 0.016*"warn" + 0.015*"countri" + 0.015*"fund" + 0.015*"claim" + 0.014*"life" + 0.013*"health" + 0.012*"say" + 0.012*"stori"

Score: 0.012506899423897266	 
Topic: 0.035*"elect" + 0.018*"water" + 0.018*"state" + 0.016*"tasmanian" + 0.013*"labor" + 0.011*"liber" + 0.011*"morrison" + 0.011*"leader" + 0.011*"parti" + 0.010*"campaign"

Score: 0.012503824196755886	 
Topic: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"climat" + 0.012*"concern" + 0.011*"flood" + 0.010*"fear" + 0.010*"emerg"

Score: 0.012503471225500107	 
Topic: 0.022*"hous" + 0.022*"south" + 0.019*"north" + 0.017*"bushfir" + 0.016*"miss" + 0.013*"interview" + 0.012*"west" + 0.012*"hospit" + 0.011*"island" + 0.011*"coast"


In [28]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4888320863246918	 Topic: 0.021*"market" + 0.020*"news" + 0.018*"women" + 0.018*"live" + 0.016*"tasmania"
Score: 0.21149252355098724	 Topic: 0.035*"elect" + 0.018*"water" + 0.018*"state" + 0.016*"tasmanian" + 0.013*"labor"
Score: 0.1829376220703125	 Topic: 0.030*"polic" + 0.029*"charg" + 0.026*"court" + 0.024*"death" + 0.024*"murder"
Score: 0.0166774932295084	 Topic: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker"
Score: 0.01667742244899273	 Topic: 0.031*"govern" + 0.019*"feder" + 0.016*"warn" + 0.015*"countri" + 0.015*"fund"
Score: 0.016676567494869232	 Topic: 0.022*"hous" + 0.022*"south" + 0.019*"north" + 0.017*"bushfir" + 0.016*"miss"
Score: 0.016676567494869232	 Topic: 0.032*"kill" + 0.023*"shoot" + 0.021*"protest" + 0.021*"dead" + 0.020*"polic"
Score: 0.016676567494869232	 Topic: 0.057*"australia" + 0.046*"australian" + 0.026*"world" + 0.018*"canberra" + 0.017*"test"
Score: 0.016676567494869232	 Topic: 0.020*"donald" + 0.014*"farmer" + 0.014*"n